In [ ]:
import math
import json
import pickle
import itertools
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from collections import defaultdict

from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
torch.multiprocessing.set_sharing_strategy('file_system')
from torch.utils.data import DataLoader
from torchvision.ops import box_iou, remove_small_boxes, nms, box_convert
from torchmetrics.detection.mean_ap import MeanAveragePrecision


from apples_detection.data.minneapple import MinneAppleDetectionModule
from apples_detection.utils.visualize import visualize_apples
from apples_detection.data.components.utils import add_leading_zeros

In [ ]:
DATA_DIR = Path("/home/dmitry/Desktop/apples-detection/data/")
MINNEAPPLE_DIR = DATA_DIR / "minneapple-detection"
CKPT_PATH = DATA_DIR / "models/best_faster_rcnn_09.04.23.ckpt"
PATCHIFIER_PATH = DATA_DIR / "minneapple-detection/test-patches_inst4_ker400max/patchifier.pkl"
PREDS_DIR = DATA_DIR / "minneapple-detection/test-patches_inst4_ker400max/preds"
GT_PATH = DATA_DIR / "minneapple-test/detection/ground_truth.json"

In [ ]:
ds_patches = MinneAppleDetectionModule(
    data_dir=MINNEAPPLE_DIR, 
    patches_suffix="-patches_inst4_ker400max", 
    flip=False,
    test_gt="../data/minneapple-test/detection/ground_truth.json",
)
# ds_patches.setup("fit")
ds_patches.setup("predict")
ds_patches.setup("test")

ds_full = MinneAppleDetectionModule(data_dir=MINNEAPPLE_DIR, flip=False)
# ds_full.setup("fit")
ds_full.setup("predict")

In [ ]:
def get_n_patches(img_shape: Tuple[int, int], strides: Tuple[int, int], kernel_size: Tuple[int, int]):
    return tuple([
            (img_shape[i] - kernel_size[i] + strides[i]) / strides[i] for i in range(2)
        ])

In [ ]:
get_n_patches([1200, 720], patchifier.strides, patchifier.kernel_size), patchifier

In [ ]:
1200 - 256 * 4

In [ ]:
with open(PATCHIFIER_PATH, "rb") as f:
    patchifier = pickle.load(f)

patches_shape = (6, 3)
total_patches = math.prod(patches_shape)

In [ ]:
def restore_size(
    strides: Tuple[int, int],
    kernel_size: Tuple[int, int],
    patches_shape: Tuple[int, int],
) -> Tuple[int, int]:
    return tuple([patches_shape[j] * strides[j] + kernel_size[j] - strides[j] for j in range(2)])

In [ ]:
def collate_fn(batch):
    if len(batch[0]) == 2:
        return tuple(zip(*batch))
    return batch

preds_files = sorted(PREDS_DIR.glob("*predictions*.pt"))
predictions = list(itertools.chain.from_iterable((torch.load(path, map_location="cpu") for path in preds_files)))
# predictions_dl = DataLoader(
#     predictions,
#     batch_size=2,
#     num_workers=1,
#     collate_fn=collate_fn
# )

# patches_dl = DataLoader(
#     ds_patches.data_val,
#     batch_size=15,
#     num_workers=3,
#     collate_fn=collate_fn,
# )

In [ ]:
def merge_boxes(
    bbox1: torch.Tensor,
    bbox2: torch.Tensor,
    policy: str = "union",
) -> torch.Tensor:
    """
    Expects bboxes in pascal_voc format: [x_min, y_min, x_max, y_max]
    """

    if policy == "union":
        return torch.Tensor([
            min(bbox1[0], bbox2[0]), min(bbox1[1], bbox2[1]),
            max(bbox1[2], bbox2[2]), max(bbox1[3], bbox2[3]),
        ])
    raise ValueError(f"Policy {policy} not supported")


def postprocess(
    boxes: torch.Tensor,
    min_size: float,
    scores: Optional[torch.Tensor] = None,
    iou_thres: float = 0.5,
):
    boxes_idx = remove_small_boxes(boxes, min_size)

    if scores is None:
        return (boxes[boxes_idx], None)
    
    boxes, scores = boxes[boxes_idx], scores[boxes_idx]
    nms_idx = nms(boxes.float(), scores, iou_thres)
    return boxes[nms_idx], scores[nms_idx]

    #     def criteria(box):
#         h, w = box[3] - box[1], box[2] - box[0]
#         squareness = min(h, w) / max(h, w)
#         return squareness
#     scores = torch.Tensor([criteria(box) for box in boxes])
#     return masks[nms(boxes, scores, 0.2), :, :]


def merge_patch_detections(
    patchifier,
    detections: List[Dict[str, torch.Tensor]],
    patches_shape: Tuple[int, int],
    min_size: float = 0.0,
    nms_iou_thres: float = 0.5,
    merge_iou_thres: float = 0.2,
):
    n_rows, n_cols = patches_shape
    strides_h, strides_w = patchifier.strides
    kernel_h, kernel_w = patchifier.kernel_size
    offsets_h = [i * strides_h for i in range(n_rows)]
    offsets_w = [j * strides_w for j in range(n_cols)]

    merged_detections = None
    for row, offset_h in enumerate(offsets_h):
        row_detections = detections[n_cols * row : n_cols * (row + 1)]
        row_merged_detections = None
        for col, (patch_detections, offset_w) in enumerate(zip(row_detections, offsets_w)):
            boxes = patch_detections["boxes"]
            if "scores" in patch_detections:
                boxes = torch.hstack([boxes, patch_detections["scores"].reshape(-1, 1)])
            
            if boxes.shape[0] == 0:
                continue

            # bboxes in format [xmin, ymin, xmax, ymax, confidence]
            # translate boxes x coords to global coords of image
            boxes[:, [0, 2]] += offset_w
            
            if row_merged_detections is None:
                row_merged_detections = boxes.clone()
                continue
            
            pairwise_iou = box_iou(row_merged_detections[:, :4], boxes[:, :4])
            
            to_merge = torch.argwhere(pairwise_iou > merge_iou_thres)
            for existing_idx, merge_idx in to_merge:
                row_merged_detections[existing_idx, :4] = merge_boxes(
                    row_merged_detections[existing_idx, :4],
                    boxes[merge_idx, :4],
                )
            
            to_append = torch.argwhere(pairwise_iou.max(dim=0).values <= merge_iou_thres).unique()
            if to_append.any():
                row_merged_detections = torch.vstack([row_merged_detections, boxes[to_append]])
        
        if row_merged_detections is None:
            continue
        
        # translate boxes y coords to global coords of image
        row_merged_detections[:, [1, 3]] += offset_h

        if merged_detections is None:
            merged_detections = row_merged_detections.clone()
            continue

        pairwise_row_iou = box_iou(merged_detections[:, :4], row_merged_detections[:, :4])

        to_merge = torch.argwhere(pairwise_row_iou > merge_iou_thres)
        for existing_idx, merge_idx in to_merge:
            merged_detections[existing_idx, :4] = merge_boxes(
                merged_detections[existing_idx, :4],
                row_merged_detections[merge_idx, :4],
            )

        to_append = torch.argwhere(pairwise_row_iou.max(dim=0).values <= merge_iou_thres).unique()
        if to_append.any():
            merged_detections = torch.vstack([merged_detections, row_merged_detections[to_append]])
    
    if merged_detections is None:
        return {
            "boxes": torch.zeros(0, 4).int(),
            "scores": torch.zeros(0).float(),
            "labels": torch.ones(0).int()
        }

    boxes = merged_detections[:, :4].int()
    scores = torch.ones(boxes.shape[0]).float() if merged_detections.shape[1] == 4 else merged_detections[:, 4].float()
    boxes, scores = postprocess(boxes, min_size, scores=scores, iou_thres=nms_iou_thres)

    merged_detections_dict = {
        "boxes": boxes,
        "labels": torch.ones(boxes.shape[0]).int(),
        "scores": scores,
    }

    return merged_detections_dict

In [ ]:
i = 0
for patch, pred in tqdm(zip(ds_patches.data_predict, predictions)):
#     cur_map = acc([pred], [gt])
    visualize_apples([(patch, pred)])
    i += 1
    
    if i >= 10:
        break

In [ ]:
min_size = 10
nms_iou_thres = 0.3
merge_iou_thres = 0.4
# merge_iou_thres = 0.2

ds = ds_full.data_predict

preds = {}
predictions_batch = []
j = 0
for patches_preds in tqdm(predictions):
    predictions_batch.append(patches_preds)

    if len(predictions_batch) == total_patches:
        merged_target = merge_patch_detections(
            patchifier, 
            predictions_batch, 
            patches_shape, 
            min_size, 
            nms_iou_thres,
            merge_iou_thres,
        )

        img_name = ds.paths[j].name
        preds[img_name] = (merged_target, j)
        j += 1

        predictions_batch.clear()

In [ ]:
with open(GT_PATH) as f:
    gt_mapping = json.load(f)

In [ ]:
annotations_by_id = defaultdict(list)
for annotation in gt_mapping["annotations"]:
    annotations_by_id[annotation["image_id"]].append(annotation["bbox"])

for img_id in annotations_by_id:
    annotations_by_id[img_id] = box_convert(torch.Tensor(annotations_by_id[img_id]).int(), in_fmt="xywh", out_fmt="xyxy")

In [ ]:
acc = MeanAveragePrecision()

for j, image_info in tqdm(enumerate(gt_mapping["images"])):
    image_name = str(add_leading_zeros(Path(image_info["filename"])))
    cur_preds, i = preds[image_name]
    image = ds[i]

    img_id = image_info["id"]
    gt_boxes = annotations_by_id[img_id]
    gt = {
        "boxes": gt_boxes,
        "scores": torch.ones(gt_boxes.shape[0]).float(),
        "labels": torch.ones(gt_boxes.shape[0]).int()
    }
    cur_map = acc([cur_preds], [gt])
    print(cur_map)
    break
#     visualize_apples([(image, gt), (image, cur_preds)])

print(acc.compute())

In [ ]:
res = {
    'map': 0.3106, 
    'map_50': 0.5941, 
    'map_75': 0.2973, 
    'map_small': 0.1834, 
    'map_medium': 0.4036, 
    'map_large': 0.0198,
}

gt = {
    "map": 0.482,
    "map_50": 0.806,
    "map_75": 0.513,
    "map_small": 0.331,
    "map_medium": 0.64,
    "map_large": 0.937,
}

In [1]:
from scripts.show_metrics import gather_metrics

In [2]:
gather_metrics("../data/metrics").iloc[:5]

,map,map_50,map_75,map_small,map_medium,map_large
model,,,,,,
gt,0.482000,0.806000,0.513000,0.331000,0.640000,0.937000
best_faster_rcnn_09.04.23,0.361082,0.699418,0.341001,0.217737,0.478240,0.525579
patches_ker400_min4_epoch11,0.344575,0.679586,0.319454,0.243952,0.451921,0.561748
patches_ker400_min4_epoch28,0.314439,0.633367,0.282121,0.214450,0.434960,0.691258
merged_patches_04.04.23,0.310600,0.594100,0.297300,0.183400,0.403600,0.019800
